In [28]:
import pandas as pd
from utils import score
from utils import input

In [29]:
nodes_input, edges = input.read_to_df("input_files/Deutschlands_Nachbarn.txt")
node_pos = score.read_to_df("result_files/Deutschlands_Nachbarn.txt.out")
node_pos.set_index("node", inplace=True)
edges = list(zip(edges.node_0.to_list(), edges.node_1.to_list()))

In [45]:
def calc_overlap(node_a, node_b):
    # ignore identical nodes
    if node_a == node_b:
        return 0
    # calculate overlap of two nodes
    node_a = node_pos.loc[node_a]
    node_b = node_pos.loc[node_b]
    dist = ((node_a.x - node_b.x)**2 + (node_a.y - node_b.y)**2)**0.5
    R = node_a.radius + node_b.radius
    if dist >= R:
        return 0
    else:
        return (R - dist) / R
    
def calc_overlap_matrix(node_list):
    # calculates the overlap matrix of a list of nodes
    overlap_matrix = pd.DataFrame(index=node_list, columns=node_list)
    for node_a in node_list:
        for node_b in node_list:
            # Scheinbar sind alle Überschneidungen ungewollt, ansonsten müsste hier eine Unterscheidung hin.
            overlap_matrix.loc[node_a, node_b] = calc_overlap(node_a, node_b)
    return overlap_matrix

In [31]:
overlap = max(calc_overlap_matrix(node_pos.index).values.flatten()) * 100

In [32]:
def calc_distance(node_a, node_b):
    # ignore identical nodes
    if node_a == node_b:
        return 0
    # calculate overlap of two nodes
    node_a = node_pos.loc[node_a]
    node_b = node_pos.loc[node_b]
    dist = ((node_a.x - node_b.x)**2 + (node_a.y - node_b.y)**2)**0.5
    R = node_a.radius + node_b.radius
    if dist >= R:
        return (dist - R) / R
    else:
        return 0
    
def calc_distance_matrix(node_list):
    # calculates the overlap matrix of a list of nodes
    distance_matrix = pd.DataFrame(index=node_list, columns=node_list)
    for node_a in node_list:
        for node_b in node_list:
            if (node_a, node_b) in edges or (node_b, node_a) in edges:
                distance_matrix.loc[node_a, node_b] = calc_distance(node_a, node_b)
            else:
                distance_matrix.loc[node_a, node_b] = -1
    return distance_matrix


In [33]:
distance = max(calc_distance_matrix(node_pos.index).values.flatten()) * 100

In [34]:
import math


def calc_angle(df, node_a, node_b):
    # ignore identical nodes
    if node_a == node_b:
        return 0
    # calculate angle of two nodes
    node_a = df.loc[node_a]
    node_b = df.loc[node_b]
    delta_x = node_a.x - node_b.x
    delta_y = node_a.y - node_b.y
    #steigung = abs(node_a.y - node_b.y) / abs(node_a.x - node_b.x)
    return math.atan2(delta_y, delta_x) / math.pi

# calc_angle_matrix can be removed
def calc_angle_matrix(nodes: pd.DataFrame):
    node_list = nodes.index
    # berechne den Winkel der Verbindungslinie zwischen den zwei Kreismittelpunkten von node_a und node_b
    angle_matrix = pd.DataFrame(index=node_list, columns=node_list)
    for node_a in node_list:
        for node_b in node_list:
            if (node_a, node_b) in edges or (node_b, node_a) in edges:
                angle_matrix.loc[node_a, node_b] = calc_angle(nodes, node_a, node_b)
            else:
                angle_matrix.loc[node_a, node_b] = -1
    return angle_matrix

def calc_angle_delta_matrix(input_nodes, output_nodes):
    node_list = input_nodes.index
    # berechne den Winkel der Verbindungslinie zwischen den zwei Kreismittelpunkten von node_a und node_b
    angle_matrix = pd.DataFrame(index=node_list, columns=node_list)
    for node_a in node_list:
        for node_b in node_list:
            if (node_a, node_b) in edges or (node_b, node_a) in edges:
                alpha_0 = calc_angle(input_nodes, node_a, node_b)
                alpha_1 = calc_angle(output_nodes, node_a, node_b)
                angle_matrix.loc[node_a, node_b] = min(abs(alpha_0 - alpha_1), 2 - abs(alpha_0 - alpha_1)) 
            else:
                angle_matrix.loc[node_a, node_b] = -1
    return angle_matrix

In [35]:
angle = max(calc_angle_delta_matrix(nodes_input, node_pos).values.flatten()) * 100

In [40]:
def calc_score(input_nodes, output_nodes, edges):
    overlap = max(calc_overlap_matrix(output_nodes.index).values.flatten()) * 100
    distance = max(calc_distance_matrix(output_nodes.index).values.flatten()) * 100
    angle = max(calc_angle_delta_matrix(input_nodes, output_nodes).values.flatten()) * 100
    n = input_nodes.shape[0]
    k = len(edges)
    print(f"n={n}, k={k}\nOverlap: {overlap}\nDistance: {distance}\nAngle: {angle}")
    return 1000 * (n+k) / (1 + 2 * overlap + distance + 0.1 * angle)

In [41]:
print(f"Score: {calc_score(nodes_input, node_pos, edges)}")

n=133, k=554
Overlap: 0
Distance: 2149.987722625085
Angle: 0.0
Score: 319.38815492706715


In [47]:
import pandas as pd
from utils import score
from utils import input
for test_case in ["Area_Afro-Eurasia", "Area_Americas", "Area_Asia", "Area_Europe", "CO2_Production_Afro-Eurasia",
                  "Deutschlands_Nachbarn", "GNI_per_capita_Afro-Eurasia","Instant_Noodle_Consumption_Eurasia",
                  "Population_Afro-Eurasia", "Population_Americas", "Population_Density_Afro-Eurasia",
                  "Population_Density_Americas"]:
    nodes_input, edges = input.read_to_df(f"input_files/{test_case}.txt")
    edges = list(zip(edges.node_0.to_list(), edges.node_1.to_list()))
    node_pos = score.read_to_df(f"result_files/{test_case}.txt.out")
    node_pos.set_index("node", inplace=True)
    print(f"\n{test_case}")
    print(f"Score: {calc_score(nodes_input, node_pos, edges)}")


Area_Afro-Eurasia
n=133, k=554
Overlap: 0
Distance: 2149.987722625085
Angle: 0.0
Score: 319.38815492706715

Area_Americas
n=23, k=74
Overlap: 0
Distance: 1109.9521339639293
Angle: 5.972759440660036
Score: 87.26557644255223

Area_Asia
n=43, k=156
Overlap: 0
Distance: 7456.242079338798
Angle: 0.0
Score: 26.68546868705709

Area_Europe
n=24, k=94
Overlap: 0
Distance: 1537.4063633535466
Angle: 0.0
Score: 76.70275085366505

CO2_Production_Afro-Eurasia
n=127, k=538
Overlap: 43.0038191106425
Distance: 7044.989896482298
Angle: 0.0
Score: 93.24175965627819

Deutschlands_Nachbarn
n=10, k=17
Overlap: 47.762402389260714
Distance: 437.78276967084776
Angle: 99.2496164187564
Score: 49.61114635651486

GNI_per_capita_Afro-Eurasia
n=126, k=534
Overlap: 0
Distance: 16611.058391312985
Angle: 0.0
Score: 39.73017578273964

Instant_Noodle_Consumption_Eurasia
n=27, k=72
Overlap: 0
Distance: 7984.951722065663
Angle: 0.0
Score: 12.396769157326242

Population_Afro-Eurasia
n=133, k=554
Overlap: 0
Distance: 25192.

In [46]:
calc_overlap_matrix(node_pos.index)

node,ECU,SUR,BRA,ARG,NIC,BLZ,BOL,HND,PRY,CAN,...,CRI,PAN,GTM,VEN,CHL,PER,USA,COL,GUF,GUY
node,,,,,,,,,,,,,,,,,,,,,
ECU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SUR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ARG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NIC,0,0,0,0,0,0,0,0.05283,0,0,...,0,0,0,0,0,0,0,0,0,0
BLZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0.159052,0,0,0,0,0,0,0
BOL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HND,0,0,0,0,0.05283,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PRY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
node_pos.index

Index(['ECU', 'SUR', 'BRA', 'ARG', 'NIC', 'BLZ', 'BOL', 'HND', 'PRY', 'CAN',
       'URY', 'SLV', 'MEX', 'CRI', 'PAN', 'GTM', 'VEN', 'CHL', 'PER', 'USA',
       'COL', 'GUF', 'GUY'],
      dtype='object', name='node')

In [50]:
import itertools

for node_a, node_b in itertools.combinations(node_pos.index, 2):
    print(node_a, node_b)

ECU SUR
ECU BRA
ECU ARG
ECU NIC
ECU BLZ
ECU BOL
ECU HND
ECU PRY
ECU CAN
ECU URY
ECU SLV
ECU MEX
ECU CRI
ECU PAN
ECU GTM
ECU VEN
ECU CHL
ECU PER
ECU USA
ECU COL
ECU GUF
ECU GUY
SUR BRA
SUR ARG
SUR NIC
SUR BLZ
SUR BOL
SUR HND
SUR PRY
SUR CAN
SUR URY
SUR SLV
SUR MEX
SUR CRI
SUR PAN
SUR GTM
SUR VEN
SUR CHL
SUR PER
SUR USA
SUR COL
SUR GUF
SUR GUY
BRA ARG
BRA NIC
BRA BLZ
BRA BOL
BRA HND
BRA PRY
BRA CAN
BRA URY
BRA SLV
BRA MEX
BRA CRI
BRA PAN
BRA GTM
BRA VEN
BRA CHL
BRA PER
BRA USA
BRA COL
BRA GUF
BRA GUY
ARG NIC
ARG BLZ
ARG BOL
ARG HND
ARG PRY
ARG CAN
ARG URY
ARG SLV
ARG MEX
ARG CRI
ARG PAN
ARG GTM
ARG VEN
ARG CHL
ARG PER
ARG USA
ARG COL
ARG GUF
ARG GUY
NIC BLZ
NIC BOL
NIC HND
NIC PRY
NIC CAN
NIC URY
NIC SLV
NIC MEX
NIC CRI
NIC PAN
NIC GTM
NIC VEN
NIC CHL
NIC PER
NIC USA
NIC COL
NIC GUF
NIC GUY
BLZ BOL
BLZ HND
BLZ PRY
BLZ CAN
BLZ URY
BLZ SLV
BLZ MEX
BLZ CRI
BLZ PAN
BLZ GTM
BLZ VEN
BLZ CHL
BLZ PER
BLZ USA
BLZ COL
BLZ GUF
BLZ GUY
BOL HND
BOL PRY
BOL CAN
BOL URY
BOL SLV
BOL MEX
BOL CRI
BOL PAN
